# 8. Generative Deep Learning

The potential of AI to emulate human thought processes goes beyond passive tasks such as object recognition and mostly ractive tasks such as driving a car. It extends well into creative activities. In 2015 there was [Google DeepDream](https://ai.googleblog.com/2015/07/deepdream-code-example-for-visualizing.html) turning an image to a psychedelic mess of dog eyes and pareidolic objects. In 2016, a short movie Sunspring was directed using a script generated by an LSTM algorithm. Other artefacts generated by a neural network include a piece in music.

A large part of artistic creation comes from simple pattern recognition and technical skill. Learning this pattern is what deep learning algorithms excel at. Machine learning models can learn the statistical <u>latent space</u> of images, music and stories, and they can <u>sample</u> from this space, creating new artworks with characteristics similar to those the model has seen in its training data.

Here, we explore from various angles the potential of deep learning to augment artistic creation. Let's get started.

### Text Generation with LSTM

Here, we will explore how recurrent neural networks can be used to generate sequence data. We'll use text generation as an example, but the same techniques can be generalized to any kind of sequence data: you can apply it to sequences of musical notes to generate new music, or timeseries of brush stroke data to generate paintings stroke by stroke, and so on.

Sequence data generation is in no way limited to artistic content generation. It has been successfully applied to speech synthesis and dialogue generation for chatbots. The Smart Reply feature from Google in 2016, capably of automatically generating a selection of quick replies to emails or text messages are powered by similar techniques.

To do so, we aim to train a network to predict the next token or next few tokens in a sequence, using the previous tokens as input. For example, given the input `the cat is on the ma`, the network is trained to predict the target `t`. Tokens are characters or words, and any network that can model the probability of the next token given the previous ones is a <b>language model</b>. A language model caputres the <u>latent space</u> of language: its statistical structure.

Once you have trained a language model, you can <u>sample</u> from it - to get new sequences. You feed it an initial string of text (called <u>conditioning data</u>) and ask it to generate the next character or the next word, add the generated output back to the input data, and repeat the process many times. For this example, we feed it strings of $N$ characters extracted from a text corpus, and train it to predit character $N+1$. The output of the model will be a softmax over all possible characters. This LSTM is called a <u>character-level neural language model</u>.

<img src="img81.png" width="750">

When generating text, the way to choose the next character is very important. There is <u>greedy sampling</u>, choosing the most likely next character. But this results in repetitive, predictable strings that don't look like coherent language. The way to get more variety is to use <u>stochastic sampling</u>. So if say `e` has a 30% chance of being the next character, it will appear in 30% of samples.

To control the amount of randomness, we use the `softmax temperature` parameter. On one extreme, each next value is equally likely to appear, resulting in maximum entropy and on the other, there is only 1 value and results in minimum entropy, so we want a sweet spot somewhere in between. So higher temperatures result in higher entropy and less predictability, and lower temperature results in lower entropy, more predictability.

<img src="img82.png" width="600">

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd

For this, let's first get a large textual corpus.

In [2]:
# Ingestion
###########
txt = ''
with open('nietzsche.txt', 'r') as f:
    txt = f.read()

In [3]:
# For testing
# print(len(t))
# print(t[:100])

In [4]:
# Preprocessing
###############
MAX_LENGTH, STEP = 60, 3

sentences, next_chars = [], []

# Iterate through the text, sampling every STEP
# Obtain the sentences and the next character from the sentence
for i in range(0, len(txt)-MAX_LENGTH, STEP):
    sentences.append(txt[i:i+MAX_LENGTH])
    next_chars.append(txt[i+MAX_LENGTH])
print('no. of sentences = {:d}'.format(len(sentences)))

# Convert the unique chars to a dictionary
unique_chars = sorted(list(set(txt)))
print('no. of unique characters = {:d}'.format(len(unique_chars)))
char_indices = dict((c, unique_chars.index(c)) for c in unique_chars)

# Vectorization
x = np.zeros((len(sentences), MAX_LENGTH, len(unique_chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(unique_chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, ch in enumerate(sentence):
        x[i, t, char_indices[ch]] = 1
    y[i, char_indices[next_chars[i]]] = 1

no. of sentences = 200281
no. of unique characters = 85


The network is a single LSTM layer, followed by a Dense classifier and softmax over all possible characters. Note that there are also other models like 1D Convnets that can do so.

In [5]:
# Instantiate model
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(MAX_LENGTH, len(unique_chars))))
model.add(keras.layers.Dense(len(unique_chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(learning_rate=0.01))

To train the model, we 

1. draw from the model a probability distribution for next character, given the generated text available so far.
2. reweight the distribution to a certain temperature
3. sample the next character at random according to the reweighted distribution
4. add the new character at the end of the available text

In [6]:
def sample(preds, temperature=1.0):
    preds_float = np.asarray(preds).astype('float64')
    preds_float = np.log(preds_float) / temperature
    preds_float = np.exp(preds_float)
    preds_float = preds_float / np.sum(preds_float)
    probs = np.random.multinomial(1, preds_float, 1)
    return np.argmax(probs)

In [7]:
print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, '[': 50, ']': 51, '_': 52, 'a': 53, 'b': 54, 'c': 55, 'd': 56, 'e': 57, 'f': 58, 'g': 59, 'h': 60, 'i': 61, 'j': 62, 'k': 63, 'l': 64, 'm': 65, 'n': 66, 'o': 67, 'p': 68, 'q': 69, 'r': 70, 's': 71, 't': 72, 'u': 73, 'v': 74, 'w': 75, 'x': 76, 'y': 77, 'z': 78, '¤': 79, '¦': 80, '©': 81, '«': 82, 'Ã': 83, '†': 84}


In [8]:
vals = []
for epoch in range(1,60):
    # Train model
    print('### epoch = %d , Training started...###' % epoch)
    model_fp = "language-model-attempt2-{:02d}.h5".format(epoch)
    m_callbacks = [keras.callbacks.ModelCheckpoint(model_fp, save_best_only=False),]
    model.fit(x, y, batch_size=128, callbacks=m_callbacks, epochs=1)
    print('### Training completed. ###')
    
    # Sample from text    
    start_index = np.random.randint(0, len(txt) - MAX_LENGTH - 1)
    generated_text = txt[start_index : start_index + MAX_LENGTH]
    print('--- Generating with seed:')
    print(generated_text)
    original_text = generated_text
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('--- temperature={:.2f} ---'.format (temperature))
        predicted_text = ''
        for i in range(400):
            sampled = np.zeros((1, MAX_LENGTH, len(unique_chars)))
            
            for t, c in enumerate(generated_text):
                sampled[0, t, char_indices[c]] = 1
            # Predict from the sample
            ypred = model.predict(sampled, verbose=0)[0]
            next_index = sample(ypred, temperature)
            next_char = unique_chars[next_index]
            predicted_text += next_char
            generated_text += next_char
            generated_text = generated_text[1:]
#         print('--- Predicted:')
#         print(original_text + predicted_text)
        vals.append((epoch, temperature, original_text, original_text + predicted_text))
        print()

### epoch = 1 , Training started...###
1565/1565 [==============================] - 181s 116ms/step - loss: 2.0297
### Training completed. ###
--- Generating with seed:
e, or somewhat feared! And pray, don't forget
the garden, th
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 2 , Training started...###
1565/1565 [==============================] - 204s 130ms/step - loss: 1.6703
### Training completed. ###
--- Generating with seed:
in a world whose essence is Will
to Power, may be reminded t
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 3 , Training started...###
1565/1565 [==============================] - 271s 173ms/step - loss: 1.5772
### Training completed. ###
--- Generating with seed:
e cases, only compulsorily, always without
delight in 'the s
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---


--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 26 , Training started...###
1565/1565 [==============================] - 283s 181ms/step - loss: 1.3360
### Training completed. ###
--- Generating with seed:
n--think
of Balzac, for instance,--unrestrained workers, alm
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 27 , Training started...###
1565/1565 [==============================] - 269s 172ms/step - loss: 1.3342
### Training completed. ###
--- Generating with seed:
nces, the entire history of the soul UP TO THE PRESENT TIME,
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 28 , Training started...###
1565/1565 [==============================] - 270s 173ms/step - loss: 1.3301
### Training completed. ###
--- Generating with seed:
g plebeianism, by which
everything is rendered opaque and le
--- temperature=0.20

/Users/bryanlim/.pyenv/versions/3.7.2/envs/botanic/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until



### epoch = 30 , Training started...###
1565/1565 [==============================] - 279s 178ms/step - loss: 1.3253
### Training completed. ###
--- Generating with seed:
consequently also, a good deal more silent. It happens more 
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 31 , Training started...###
1565/1565 [==============================] - 280s 179ms/step - loss: 1.3238
### Training completed. ###
--- Generating with seed:
n that account, as instruments, they are
far from being phil
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 32 , Training started...###
1565/1565 [==============================] - 282s 180ms/step - loss: 1.3224
### Training completed. ###
--- Generating with seed:
 SWEAT of the
noble"--but not at all as something easy and d
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20


--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 55 , Training started...###
1565/1565 [==============================] - 192s 123ms/step - loss: 1.2879
### Training completed. ###
--- Generating with seed:
ity and he
soothes himself a little with the assertion that 
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 56 , Training started...###
1565/1565 [==============================] - 199s 127ms/step - loss: 1.2877
### Training completed. ###
--- Generating with seed:
 a phenomenon as the
loftiest heroism of morality. It is alw
--- temperature=0.20 ---

--- temperature=0.50 ---

--- temperature=1.00 ---

--- temperature=1.20 ---

### epoch = 57 , Training started...###
1565/1565 [==============================] - 239s 153ms/step - loss: 1.2865
### Training completed. ###
--- Generating with seed:
m
attained by a German, or almost always too late. The maste
--- temperature=0.20

As you can see, a low temperature value results in extremely repetitive and predictable text, but local structure is highly realistic. Most of the time the words are real English words. With higher temperatures, the generated text becomes more interesting, surprising, even creative. It sometimes invents completely new owrds that sound somewhat plausible. A good balance between learned structure and randomness is what makes generation interesting.

In [9]:
# # Train model
# vals = []
# epoch = 2
# model_fp = "language-model-{:02d}.h5".format(epoch)
# model = keras.models.load_model(model_fp)
# # Sample from text    
# start_index = np.random.randint(0, len(txt) - MAX_LENGTH - 1)
# generated_text = txt[start_index : start_index + MAX_LENGTH]
# print('--- Generating with seed:')
# print(generated_text)
# original_text = generated_text

# for temperature in [0.2, 0.5, 1.0, 1.2]:
#     print('--- temperature={:.2f} ---'.format (temperature))
#     predicted_text = ''
#     for i in range(400):
#         sampled = np.zeros((1, MAX_LENGTH, len(unique_chars)))

#         for t, c in enumerate(generated_text):
#             sampled[0, t, char_indices[c]] = 1
#         # Predict from the sample
#         ypred = model.predict(sampled, verbose=0)[0]
#         next_index = sample(ypred, temperature)
#         next_char = unique_chars[next_index]
#         predicted_text += next_char
#         generated_text += next_char
#         generated_text = generated_text[1:]
#     print('--- Predicted:')
#     print(original_text + predicted_text)
#     vals.append((epoch, temperature, original_text, original_text + predicted_text))
#     print()

In [10]:
results_df = pd.DataFrame(vals, columns=['epoch', 'temperature', 'seed', 'result'])

In [11]:
results_df.to_csv('language_model_results.csv', sep='|', index=False)